## Accessing Data from the MAAP

In this example, we demonstrate how to access data from the MAAP using the 'requests' method of python. At this time, this procedure is the same for user-contributed data added to the store.

We import the `os` module, import the `MAAP` package, and create a new MAAP class.

In [41]:
# import os module
import os

# import the MAAP package
from maap.maap import MAAP

# create MAAP class
maap = MAAP()

In [42]:
SHORTNAME = "AFLVIS2"

# search for granules with SHORTNAME
results = maap.searchGranule(short_name=SHORTNAME)


We assign a variable (in this case, `data_file`) to the first result of our search from the cell above.

In [43]:
# grab first result
data_file = results[0]
data_file

{'concept-id': 'G1549411275-NSIDC_ECS',
 'collection-concept-id': 'C1549378743-NSIDC_ECS',
 'revision-id': '3',
 'format': 'application/echo10+xml',
 'Granule': {'GranuleUR': 'SC:AFLVIS2.001:138345559',
  'InsertTime': '2018-09-24T10:55:46.870Z',
  'LastUpdate': '2018-09-24T10:56:29.264Z',
  'Collection': {'DataSetId': 'AfriSAR LVIS L2 Geolocated Surface Elevation Product V001'},
  'DataGranule': {'SizeMBDataGranule': '130.082',
   'ProducerGranuleId': 'LVIS2_Gabon2016_0220_R1808_038024.TXT',
   'DayNightFlag': 'UNSPECIFIED',
   'ProductionDateTime': '2018-08-23T18:27:43.000Z',
   'LocalVersionId': '001'},
  'Temporal': {'RangeDateTime': {'BeginningDateTime': '2016-02-20T10:33:44.015000Z',
    'EndingDateTime': '2016-02-20T10:43:39.524000Z'}},
  'Spatial': {'HorizontalSpatialDomain': {'Geometry': {'GPolygon': {'Boundary': {'Point': [{'PointLongitude': '9.78612',
         'PointLatitude': '0.17149'},
        {'PointLongitude': '9.84714', 'PointLatitude': '0.14268'},
        {'PointLongi

A data directory is then set, and if the directory does not already exist, it is created. The file from our search is then downloaded into the file system in this directory. Here, the function `requests` method from python to download the URL.

In [56]:
# set data directory
dataDir = './data'

# check if directory exists -> if directory doesn't exist, directory is created
if not os.path.exists(dataDir):
    os.mkdir(dataDir)
# extract the link to the resource 
data_url = data_file['Granule']['OnlineAccessURLs']['OnlineAccessURL'][0]['URL']


Now we can see that the data directory has been created and the 'LVIS2_Gabon2016_0308_R1808_038002.TXT' file is downloaded into the directory. The downloaded file remains in the data directory until the user deletes it.

In [57]:
import requests
r = requests.get(data_url)

from pathlib import Path
filename = Path(data_url).name
with open(f'./{dataDir}/{filename}', 'wb') as f:
    f.write(r.content)

## Accessing Data from AWS Requester Pays Buckets

Some data is cloud available but in requester pays buckets. In this example, we use Rasterio, Boto3, and MAAP's `aws.requester_pays_credentials()` function to retrieve data within the `usgs-landsat` requester pays bucket.

In [ ]:
import boto3
import rasterio as rio

from maap.maap import MAAP
from rasterio.plot import show
from rasterio.session import AWSSession


maap = MAAP(maap_host='api.ops.maap-project.org')
credentials = maap.aws.requester_pays_credentials()

boto3_session = boto3.Session(
    aws_access_key_id=credentials['aws_access_key_id'],
    aws_secret_access_key=credentials['aws_secret_access_key'],
    aws_session_token=credentials['aws_session_token']
)

aws_session = AWSSession(boto3_session, requester_pays=True)
file_s3 = 's3://usgs-landsat/collection02/level-2/standard/oli-tirs/2015/044/025/LC08_L2SP_044025_20150812_20200908_02_T1/LC08_L2SP_044025_20150812_20200908_02_T1_SR_B2.TIF'
with rio.Env(aws_session):
    with rio.open(file_s3, 'r') as src:
        # list of overviews
        oviews = src.overviews(1)
        # get second item from list to retrieve a thumbnail
        oview = oviews[1]
        # read first band of file and set shape of new output array
        thumbnail = src.read(1, out_shape=(1, int(src.height // oview), int(src.width // oview)))
# now display the thumbnail
show(thumbnail)

You may adjust the expiration time of the AWS credentials generated by `maap.aws.requester_pays_credentials()`: 

In [ ]:
# Credential expiration time in seconds (defaults to 12 hours)
maap.aws.requester_pays_credentials(expiration=3600)